In [4]:
import psycopg2
from psycopg2 import sql
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
import os
from dotenv import load_dotenv

# Load the OpenAI API key from the environment
load_dotenv()
api_key = os.environ.get("OPENAI_API_KEY")

# Initialize the OpenAI object
llm = OpenAI(openai_api_key=api_key)

# Function to get the schema from the database
def get_schema():
    # Connect to your database
    conn = psycopg2.connect(database="youtube_data", user="postgres", password="postgres", host="localhost", port="15432")
    cur = conn.cursor()

    # Query the information_schema to get the schema
    cur.execute("""
        SELECT table_name, column_name, data_type
        FROM information_schema.columns
        WHERE table_schema = 'youtube_data_schema' AND table_name = 'youtube_info'
    """)
    # 
    # # Format the schema into a string
    # schema = "We have a database named youtube_data with the following tables:\n"
    # for row in cur.fetchall():
    #     schema += f"- {row[0]}: Contains information about the {row[0]}. Columns: {row[1]} ({row[2]}).\n"
    # Format the schema into a string
    schema = "The database named youtube_data has the following tables:\n"
    
    # Initialize an empty dictionary to store table information
    tables = {}
    
    # Fetch all rows and organize them by table
    for row in cur.fetchall():
        table_name, column_name, data_type = row
        if table_name not in tables:
            tables[table_name] = []
        tables[table_name].append(f"{column_name} ({data_type})")
    
    # Add table information to the schema string
    for table_name, columns in tables.items():
        schema += f"- {table_name}: Contains information about the {table_name}. Columns: {', '.join(columns)}.\n"


    return schema


In [5]:

# Get the schema string
schema = get_schema()
print(schema)

youtube_data database has the following tables:
- youtube_info: Contains information about the youtube_info. Columns: date (text), Content type_Other (integer), Content type_Videos (integer), Device type_Computer (integer), Device type_Mobile phone (integer), Device type_TV (integer), Device type_Tablet (integer), Geography_Unnamed: 1 (integer), geography_bj (integer), geography_br (integer), geography_ch (integer), geography_de (integer), geography_es (integer), geography_et (integer), geography_fr (integer), geography_gb (integer), geography_gh (integer), geography_hu (integer), geography_id (integer), geography_in (integer), geography_it (integer), geography_jp (integer), geography_ke (integer), geography_kr (integer), geography_lk (integer), geography_ng (integer), geography_nl (integer), geography_ph (integer), geography_ro (integer), geography_rs (integer), geography_rw (integer), geography_sd (integer), geography_sn (integer), geography_tg (integer), geography_us (integer), geog

In [8]:
from langchain_community.llms.fake import FakeStreamingListLLM
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import SystemMessagePromptTemplate
from langchain_core.runnables import Runnable
from operator import itemgetter

prompt = (
    SystemMessagePromptTemplate.from_template("You are a nice assistant.")
    + "{question}"
)

prompt

ChatPromptTemplate(input_variables=['question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a nice assistant.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))])

In [31]:
from langchain_core.prompts import SystemMessagePromptTemplate

sql_query_example = "SELECT 'date', 'Content_type_Videos', 'Device type_Mobile phone' FROM youtube_data_schema.youtube_info WHERE 'geography_us' > 0"

# Define the system message
system_message = f"You are a nice assistant. You are trained to generate SQL queries for Redash based on user's questions. An example is this: {sql_query_example}. But it must note be exact like that. Just note that the table names are enclosed in double quotations and the part after 'FROM'. \nYou are not to make up any information, if you don't know, say 'I don't know'. You have access to the youtube database with the following schema:\n" + schema

# Create a SystemMessagePromptTemplate from the system message
prompt = (
    SystemMessagePromptTemplate.from_template(system_message) + "{question}"
)

# Define the user's question
question = "What are the youtube views from TV in germany"


# Chain the full PromptTemplate and the OpenAI object together
llm_chain = prompt | llm

# Invoke the chain to get a response from the model
response = llm_chain.invoke(question)

# The response variable now contains the response from the OpenAI API. I can send this response back to the user.
print(response)


?

System: SELECT "date", "Device type_TV", "geography_de" FROM youtube_data_schema.youtube_info WHERE "geography_de" > 0


In [3]:
# Create a PromptTemplate from the schema
schema_prompt = PromptTemplate.from_template(schema)

# Chain the schema PromptTemplate and the OpenAI object together
llm_chain = schema_prompt | llm

# Now you can use this chain to ask questions to the model
question = "What is the average price of the products ordered by user with id 1?"
response = llm_chain.invoke(question)

# The response variable now contains the response from the OpenAI API. You can send this response back to the user.

TypeError: Expected mapping type as input to PromptTemplate. Received <class 'str'>.